In [1]:
from transformers import XLNetTokenizer, XLNetLMHeadModel
import torch

tokenizer = XLNetTokenizer.from_pretrained('xlnet-large-cased')
model = XLNetLMHeadModel.from_pretrained('xlnet-large-cased')

In [2]:
PADDING_TEXT = """Python is an interpreted high-level general-purpose programming language. Its design philosophy emphasizes code readability with its use of significant indentation. Its language constructs as well as its object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.[30]

Python is dynamically-typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly, procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.[31]

Guido van Rossum began working on Python in the late 1980s, as a successor to the ABC programming language, and first released it in 1991 as Python 0.9.0.[32] Python 2.0 was released in 2000 and introduced new features, such as list comprehensions and a garbage collection system using reference counting. Python 3.0 was released in 2008 and was a major revision of the language that is not completely backward-compatible. Python 2 was discontinued with version 2.7.18 in 2020.[33]

Python consistently ranks as one of the most popular programming languages. <eod> </s> <eos>"""

In [9]:
torch.manual_seed(0)
# We show how to setup inputs to predict a next token using a bi-directional context.
# We will predict masked tokens
input_ids = torch.tensor(tokenizer.encode(PADDING_TEXT, add_special_tokens=False)).unsqueeze(0)  

targets = [ -6, -4]

perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
perm_mask[0, :, targets] = 1.0  # Previous tokens don't see last token

target_mapping = torch.zeros((1, len(targets), input_ids.shape[1]), dtype=torch.float)  

target_mapping[0, 0, targets[0]] = 1.0  # Our first  prediction 
target_mapping[0, 1, targets[1]] = 1.0  # Our second  prediction 

input_ids_tensor = input_ids.to("cpu")
target_mapping_tensor = target_mapping.to("cpu")
perm_mask_tensor = perm_mask.to("cpu")

model.eval()
if torch.cuda.is_available(): model.to('cuda') #if we have a GPU 

with torch.no_grad():
  outputs = model(input_ids_tensor, perm_mask=perm_mask_tensor, target_mapping=target_mapping_tensor)
next_token_logits = outputs[0]  # Output has shape [target_mapping.size(0), target_mapping.size(1), config.vocab_size]

for j in range(len(targets)):
  predicted_k_indexes = torch.topk(outputs[0][0][j],k=5)
  predicted_logits_list = predicted_k_indexes[0] 
  predicted_indexes_list = predicted_k_indexes[1] 
    
  print ("predicted word:",tokenizer.decode(input_ids[0][targets[j]].item()), j)
  for i,item  in enumerate(predicted_indexes_list):
      the_index = predicted_indexes_list[i].item()
      print("word and logits",tokenizer.decode(the_index),predicted_logits_list[i].item())

predicted word: <eod> 0
word and logits <eop> -0.9852077960968018
word and logits < -4.28443717956543
word and logits < -4.850223541259766
word and logits By -4.871249675750732
word and logits by -5.666018486022949
predicted word: < 1
word and logits . 8.380627632141113
word and logits  8.354589462280273
word and logits s 8.143485069274902
word and logits / 7.687319278717041
word and logits </ 7.284868240356445


In [10]:
import random
import numpy as np

# Function to select topK tokens from the probability list and 
# then based on the selected K word distribution get sample of random token IDs
def choose_from_top(probs, k=5, sample_size=1):
    ind = np.argpartition(probs, -k)[-k:]
    top_prob = probs[ind]
    # print(tokenizer.decode(ind))
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(k, sample_size, p = top_prob, replace=False)
    token_ids = ind[choice]
    return token_ids

sent = "python is a "
topk = 10
n = 20
# Lower temperatures make the model more confident in its top choices, while temperatures greater than 1 decrease confidence.
temperature = 5

model.eval()
if torch.cuda.is_available(): model.to('cuda') #if we have a GPU 

sent_tokens = tokenizer.encode(sent, add_special_tokens=False)
mask_tokens = tokenizer.encode('<mask>', add_special_tokens=False)
padding_tokens = tokenizer.encode(PADDING_TEXT, add_special_tokens=False)
   
for i in range(n):
  input = mask_tokens + sent_tokens + mask_tokens     
  target_id1 = -len(input)
  target_id2 = -1

  input_ids = torch.tensor(padding_tokens + input).unsqueeze(0)   # We will predict masked tokens

  perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
  perm_mask[0, :, [target_id1, target_id2]] = 1.0  # Previous tokens don't see last token

  target_mapping = torch.zeros((1, 2, input_ids.shape[1]), dtype=torch.float)  
  target_mapping[0, 0, target_id1] = 1.0  # Our first  prediction 
  target_mapping[0, 1, target_id2] = 1.0  # Our second  prediction 

  input_ids_tensor = input_ids.to("cpu")
  target_mapping_tensor = target_mapping.to("cpu")
  perm_mask_tensor = perm_mask.to("cpu")

  with torch.no_grad():
    outputs = model(input_ids_tensor, perm_mask=perm_mask_tensor, target_mapping=target_mapping_tensor)

  predicted_tokens = []
  
  for j in range(2):
    probs = torch.nn.functional.softmax(outputs[0][0][j]/temperature, dim = 0).to('cpu').numpy()
    predicted_tokens.append(choose_from_top(probs, k=topk, sample_size=1))

  if i % 2 == 0:    
    tok = predicted_tokens[0][0]
    sent_tokens = [tok] + sent_tokens 
    print('left: ', tokenizer.decode(sent_tokens))
  else:     
    tok = predicted_tokens[1][0]
    sent_tokens = sent_tokens + [tok]
    print("right: ", tokenizer.decode(sent_tokens)) 

left:  <eop> python is a
right:  <eop> python is a language
left:  |<eop> python is a language
right:  |<eop> python is a language that
left:  /|<eop> python is a language that
right:  /|<eop> python is a language that uses
left:  /|<eop> python is a language that uses
right:  /|<eop> python is a language that uses an
left:  <eop> /|<eop> python is a language that uses an
right:  <eop> /|<eop> python is a language that uses an interpreted
left:  ><eop> /|<eop> python is a language that uses an interpreted
right:  ><eop> /|<eop> python is a language that uses an interpreted high
left:  br><eop> /|<eop> python is a language that uses an interpreted high
right:  br><eop> /|<eop> python is a language that uses an interpreted high
left:  /br><eop> /|<eop> python is a language that uses an interpreted high
right:  /br><eop> /|<eop> python is a language that uses an interpreted high level
left:  /br><eop> /|<eop> python is a language that uses an interpreted high level
right:  /br><eop> /|<eo

In [11]:
import random

padding_tokens = tokenizer.encode(PADDING_TEXT, add_special_tokens=False)
mask_tokens = tokenizer.encode('<mask>', add_special_tokens=False)

model.eval()
if torch.cuda.is_available(): model.to('cuda') #if we have a GPU 

def candidates_gen(sent_tokens, candidate=([], 1, []), d='left', n_candidates=5, topk=20, temperature=5):
  branch_candidates = []  
  cand_tokens = candidate[0]
  
  if d == 'right':    
    input = sent_tokens + cand_tokens + mask_tokens     
    
    target_id = -1
    input_ids = torch.tensor(padding_tokens + input).unsqueeze(0)  

    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[0, :, target_id] = 1.0  # Previous tokens don't see last token
  else:        
    input = mask_tokens + cand_tokens + sent_tokens    
    
    target_id = -len(input)  
    input_ids = torch.tensor(padding_tokens + input).unsqueeze(0)  

    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[0, :, [target_id - i for i in range(100)]] = 1.0  # Mask additional previos tokens to improve left-side generation

  # We will predict masked tokens 
  target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)  
  target_mapping[0, 0, target_id] = 1.0  # Our right  prediction 

  if torch.cuda.is_available():
    input_ids_tensor = input_ids.to("cuda")
    target_mapping_tensor = target_mapping.to("cuda")
    perm_mask_tensor = perm_mask.to("cuda")
  else:
    input_ids_tensor = input_ids
    target_mapping_tensor = target_mapping
    perm_mask_tensor = perm_mask

  with torch.no_grad():
    outputs = model(input_ids_tensor, perm_mask=perm_mask_tensor, target_mapping=target_mapping_tensor)

  probs = torch.nn.functional.softmax(outputs[0][0][0]/temperature, dim = 0)
  selected_indexes = choose_from_top(probs.to('cpu').numpy(), k=topk, sample_size=n_candidates)
  selected_probs = probs[selected_indexes]

  for i,item  in enumerate(selected_indexes):
      the_index = item.item()
      if d == "right":
        new_sent = cand_tokens + [the_index]
      elif d == "left":
        new_sent = [the_index] + cand_tokens
      
      prob = selected_probs[i].item()
      # add word combinations to branch_candidates in format [sentence, cumulative probability, all probs]
      branch_candidates.append((new_sent, candidate[1] * prob, candidate[2] + [prob]))
  
  return branch_candidates

In [12]:
def beam_gen(sent_tokens, candidates, depth=5, d='right', sample_size=2, topk=10, temperature=5):
  beams = candidates[:]
  new_candidates = candidates[:]
  while depth > 0:
    new_candidates = []
    for candidate in candidates:
      for new_candidate in candidates_gen(sent_tokens, candidate, d, sample_size, topk, temperature):
        beams.append(new_candidate)
        new_candidates.append(new_candidate)   
    print("Number of beams:", len(new_candidates))    
    candidates = new_candidates[:]
    depth -= 1
  # sort candidate beams by a sum of logaryphms of probability of each word in a beam. Which is equivalet to product of probabilities 
  sorted_beams = sorted(new_candidates, key=lambda tup: np.sum(np.log10(tup[2])), reverse=True)
  return beams, sorted_beams

In [13]:
def bi_generator(sent, direction, first_sample_size, sample_size, n_tokens, topk, iterations, temperature):
  sent_tokens = tokenizer.encode(sent, add_special_tokens=False) 

  for i in range(iterations):
    if (i % 2 == 0 and direction == 'both') or direction == 'left':
      print('>> left side generation')
      candidates = candidates_gen(sent_tokens=sent_tokens, d='left', n_candidates=first_sample_size,  topk=topk, temperature=temperature)
      beams, sorted_beams = beam_gen(sent_tokens, candidates, n_tokens-1, 'left', sample_size, topk, temperature=temperature)
      topn = len(sorted_beams)//5 if len(sorted_beams) > 4 else len(sorted_beams)
      selected_candidate = random.choice(sorted_beams[:topn])
      sent_tokens = selected_candidate[0] + sent_tokens
      print(tokenizer.decode(sent_tokens))
    if (i % 2 != 0 and direction == 'both') or direction == 'right':
      print('>> right side generation')
      candidates = candidates_gen(sent_tokens=sent_tokens, d='right', n_candidates=first_sample_size, topk=topk, temperature=temperature)
      beams, sorted_beams = beam_gen(sent_tokens, candidates, n_tokens-1, 'right', sample_size, topk, temperature=temperature)
      topn = len(sorted_beams)//5 if len(sorted_beams) > 4 else len(sorted_beams)
      selected_candidate = random.choice(sorted_beams[:topn])
      sent_tokens = sent_tokens + selected_candidate[0]
      print(tokenizer.decode(sent_tokens))
    
  return tokenizer.decode(sent_tokens)

In [14]:
sent = "python is a "  
first_sample_size = 4
sample_size = 2
n_tokens = 4
topk = 20
iterations = 6
temperature = 4
direction = "both"

bi_generator(sent, direction, first_sample_size, sample_size, n_tokens, topk, iterations, temperature);


>> left side generation
Number of beams: 8
Number of beams: 16
Number of beams: 32
.pg) python is a
>> right side generation
Number of beams: 8
Number of beams: 16
Number of beams: 32
.pg) python is a Java script generator.
>> left side generation
Number of beams: 8
Number of beams: 16
Number of beams: 32
ttm.pg) python is a Java script generator.
>> right side generation
Number of beams: 8
Number of beams: 16
Number of beams: 32
ttm.pg) python is a Java script generator. One of many Python
>> left side generation
Number of beams: 8
Number of beams: 16
Number of beams: 32
.bg) ttm.pg) python is a Java script generator. One of many Python
>> right side generation
Number of beams: 8
Number of beams: 16
Number of beams: 32
.bg) ttm.pg) python is a Java script generator. One of many Python languages. One of


In [ ]:
from transformers import XLNetTokenizer, XLNetModel
import torch

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetModel.from_pretrained('xlnet-base-cased')

In [ ]:
sent = 'a', 'h', 'd'
sent_tokens = tokenizer.encode(sent)
sent_tokens